<a href="https://colab.research.google.com/github/sindhu819/Projects/blob/master/Yelp_for_Business.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initiating the required modules in notebook

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 51kB/s 
     |████████████████████████████████| 204kB 49.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=847e9abc40e3eb9998079e72124e137b4352000df12daca204803492f91c56e5
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
!pip install -q findspark

In [0]:
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

from pyspark.ml.linalg import DenseVector

from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.sql.types import *

from operator import add
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml.clustering import LDA, LDAModel, LocalLDAModel
from wordcloud import WordCloud


In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# Mounting the local drive which has our data files

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Loading the reviews paraquet files

In [0]:
file1=r'drive/My Drive/Dataset/Yelp_Reviews.parquet'
reviews=spark.read.parquet(file1)

In [0]:
reviews.count()

1076129

In [0]:
reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



#Loading Business file

In [0]:
file2=r'drive/My Drive/Dataset/Yelp_Business.csv'
business=spark.read.csv(file2,header=True)

In [0]:
# check the dataframe schema
business.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- business_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- categories: string (nullable = true)



In [0]:
business.createOrReplaceTempView("businesses")
reviews.createOrReplaceTempView("reviews")

## Selecting required columns from business file

In [0]:
query = """
SELECT business_id, business_name as Name,Review_count,stars as Rating,categories FROM businesses
"""

In [0]:
tmp = spark.sql(query).limit(10).toPandas()
tmp.head()

,business_id,Name,Review_count,Rating,categories
0,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,40,4.0,"Restaurants, Italian"
1,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,184,4.5,"Mexican, Restaurants, Patisserie/Cake Shop, Fo..."
2,kANF0dbeoW34s2vwh6Umfw,Dairy Queen,33,2.0,"Fast Food, Food, Restaurants, Ice Cream & Froz..."
3,X8mtoSxY8whtmbDlj0D3Aw,Imperial Asian Buffet,4,2.0,"Restaurants, Chinese"
4,bJP4l_BGq2CudEu0m-wNjg,Artisan Fine Dining Room,3,2.0,"Restaurants, Pizza, Italian, American (New)"


In [0]:
business_df = spark.sql(query)

In [0]:
business_df.show(5)

+--------------------+--------------------+------------+------+--------------------+
|         business_id|                Name|Review_count|Rating|          categories|
+--------------------+--------------------+------------+------+--------------------+
|PZ-LZzSlhSe9utkQY...|Carluccio's Tivol...|          40|   4.0|Restaurants, Italian|
|tstimHoMcYbkSC4eB...|Maria's Mexican R...|         184|   4.5|Mexican, Restaura...|
|kANF0dbeoW34s2vwh...|         Dairy Queen|          33|   2.0|Fast Food, Food, ...|
|X8mtoSxY8whtmbDlj...|Imperial Asian Bu...|           4|   2.0|Restaurants, Chinese|
|bJP4l_BGq2CudEu0m...|Artisan Fine Dini...|           3|   2.0|Restaurants, Pizz...|
+--------------------+--------------------+------------+------+--------------------+
only showing top 5 rows



In [0]:
query = """
SELECT business_id, review_text, review_date as dates, stars as rating  FROM reviews
"""

In [0]:
reviews_df = spark.sql(query)

In [0]:
import string
reviews_df.select('review_text').take(1)

[Row(review_text="The food is always good and the prices are reasonable.  Although it's not exactly a restaurant -- more like a snack bar, limited menu and you walk up to the counter, place your order, wait a minute or so for them to prepare your order, and pay the cashier, then search for a table.  Kind of like a sandwich shop.  Still a much better choice for value than the other hotel Deli's on the strip.")]

In [0]:
reviews_df.take(1)

[Row(business_id='I4Nr-MVc26qWr08-S3Q1ow', review_text="The food is always good and the prices are reasonable.  Although it's not exactly a restaurant -- more like a snack bar, limited menu and you walk up to the counter, place your order, wait a minute or so for them to prepare your order, and pay the cashier, then search for a table.  Kind of like a sandwich shop.  Still a much better choice for value than the other hotel Deli's on the strip.", dates='2016-01-17 05:26:22', rating=4.0)]

In [0]:
import string
from pyspark.sql.functions import udf
customudf = udf(lambda s: s.translate(str.maketrans('', '', string.punctuation)), StringType())

In [0]:
reviewstmp =reviews_df.withColumn("reviews_text",customudf(reviews_df.review_text))

In [0]:
reviewstmp.take(1)

[Row(business_id='I4Nr-MVc26qWr08-S3Q1ow', review_text="The food is always good and the prices are reasonable.  Although it's not exactly a restaurant -- more like a snack bar, limited menu and you walk up to the counter, place your order, wait a minute or so for them to prepare your order, and pay the cashier, then search for a table.  Kind of like a sandwich shop.  Still a much better choice for value than the other hotel Deli's on the strip.", dates='2016-01-17 05:26:22', rating=4.0, reviews_text='The food is always good and the prices are reasonable  Although its not exactly a restaurant  more like a snack bar limited menu and you walk up to the counter place your order wait a minute or so for them to prepare your order and pay the cashier then search for a table  Kind of like a sandwich shop  Still a much better choice for value than the other hotel Delis on the strip')]

### Taking the top review counts - 2680 restaurants out of 5909 total restaurants in las vegas 

Doing this to restrict total number of reviews to 1 Million -- As Auto ML (Amazon comprehend) can process only 1M records at a time

In [0]:
business_list = reviews_df.groupBy("business_id").agg(count("review_text")).orderBy(desc("count(review_text)")).select("business_id").limit(2680)

In [0]:
filtered_business = business_df.filter(business_df['categories'].isin(['Restaurants']))

In [0]:
filtered_business.createOrReplaceTempView("filtered_business")

In [0]:
filtered_business.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Review_count: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- categories: string (nullable = true)



In [0]:
business_pandas = business_list.limit(10000000).toPandas()

In [0]:
blist = list(business_pandas['business_id'])

In [0]:
filtered_reviews = reviews_df.filter(reviews_df['business_id'].isin(blist))

In [0]:
filtered_reviews.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- dates: string (nullable = true)
 |-- rating: double (nullable = true)



In [0]:
filtered_reviews.count()

999453

### Add index column for easy interpretation of comprehend 

In [0]:
from pyspark.sql.window import Window  
w = Window.orderBy("business_id") 
filtered_reviews = filtered_reviews.withColumn("id", row_number().over(w)-1)
filtered_reviews.filter(filtered_reviews['id']==0).show()

### Write filtered_reviews to a seperate file

In [0]:
filtered_reviews.coalesce(1).write.option("header","true").csv("drive/My Drive/Data Mining/restaurantstotal.csv")

### Tokenize and join to properly remove unnecessary characters in reviews

In [0]:
def tokenize(c):
   return split(regexp_replace(lower(c), "[^a-zA-Z0-9\\s]", ""), "\\s+")
res = filtered_reviews.withColumn('review_text',tokenize(filtered_reviews.review_text))

In [0]:
res.take(1)

[Row(business_id='I4Nr-MVc26qWr08-S3Q1ow', review_text=['the', 'food', 'is', 'always', 'good', 'and', 'the', 'prices', 'are', 'reasonable', 'although', 'its', 'not', 'exactly', 'a', 'restaurant', 'more', 'like', 'a', 'snack', 'bar', 'limited', 'menu', 'and', 'you', 'walk', 'up', 'to', 'the', 'counter', 'place', 'your', 'order', 'wait', 'a', 'minute', 'or', 'so', 'for', 'them', 'to', 'prepare', 'your', 'order', 'and', 'pay', 'the', 'cashier', 'then', 'search', 'for', 'a', 'table', 'kind', 'of', 'like', 'a', 'sandwich', 'shop', 'still', 'a', 'much', 'better', 'choice', 'for', 'value', 'than', 'the', 'other', 'hotel', 'delis', 'on', 'the', 'strip'], dates='2016-01-17 05:26:22', rating=4.0)]

In [0]:
join_udf = udf(lambda x: " ".join(x))
out = res.withColumn("review_text", join_udf(col("review_text")))

In [0]:
out.take(1)

[Row(business_id='I4Nr-MVc26qWr08-S3Q1ow', review_text='the food is always good and the prices are reasonable although its not exactly a restaurant more like a snack bar limited menu and you walk up to the counter place your order wait a minute or so for them to prepare your order and pay the cashier then search for a table kind of like a sandwich shop still a much better choice for value than the other hotel delis on the strip', dates='2016-01-17 05:26:22', rating=4.0)]

In [0]:
out2 = out.select('review_text')

In [0]:
out2.count()

999453

### out2 has only reviews column -- Input for Amazon comprehend

In [0]:
out2.coalesce(1).write.option("header","true").csv("drive/My Drive/Data Mining/restauranttokens2.csv")

###### Next Steps:
###### -Upload to S3 
###### -Submit reviews(out2) as input to comprehend job 
###### -Download the output and interpret the topics (Human Intervension)

Interpreted topics based on the output words :   
Varieties   
Ambience   
Management   
Service   
Value  

Intension behind interpretation is to assign words which are also common deciding factors for telling if a restaurant is good/bad

##### Code to upload file in amazon s3. Filenamehere we give the above  saved file (out2)

In [0]:
import logging
import boto3
from botocore.exceptions import ClientError

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3',aws_access_key_id='*******', aws_secret_access_key='******')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

# Auto ML (Amazon Comprehend)  -- Job time: 35 mins for 1M reviews.

---- If productionized or scaled to other regions. We should submit Amazon comprehend jobs for all the reviews in each region and work on corresponding outputs, Region wise. (Deploy Region wise Topic modelling Jobs in Amazon Comprehend)

In [0]:
import boto3
import json
from bson import json_util
 
comprehend = boto3.client(aws_access_key_id='*******',aws_secret_access_key='*****', service_name='comprehend', region_name='us-west-2')
                
input_s3_url = "s3://yelpsuggest/las_vegas_reviews.csv"
input_doc_format = "ONE_DOC_PER_LINE"
output_s3_url = "s3://yelpsuggest/las_vegas_topics.csv"
data_access_role_arn = "arn:aws:iam::844760149411:role/yelp+suggest"
number_of_topics = 5
 
input_data_config = {"S3Uri": input_s3_url, "InputFormat": input_doc_format}
output_data_config = {"S3Uri": output_s3_url}
 
start_topics_detection_job_result = comprehend.start_topics_detection_job(NumberOfTopics=number_of_topics,
                                                                              InputDataConfig=input_data_config,
                                                                              OutputDataConfig=output_data_config,
                                                                              DataAccessRoleArn=data_access_role_arn)
 
print('start_topics_detection_job_result: ' + json.dumps(start_topics_detection_job_result))
 
job_id = start_topics_detection_job_result["JobId"]
 
print('job_id: ' + job_id)
 
describe_topics_detection_job_result = comprehend.describe_topics_detection_job(JobId=job_id)
 
print('describe_topics_detection_job_result: ' + json.dumps(describe_topics_detection_job_result, default=json_util.default))
 
list_topics_detection_jobs_result = comprehend.list_topics_detection_jobs()
 
print('list_topics_detection_jobs_result: ' + json.dumps(list_topics_detection_jobs_result, default=json_util.default))


start_topics_detection_job_result: {"JobId": "34fa6597931e583f4d05bfee08bc0c8d", "JobStatus": "SUBMITTED", "ResponseMetadata": {"RequestId": "03a28c82-1dfa-480c-8656-687cd9c8fd95", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amzn-requestid": "03a28c82-1dfa-480c-8656-687cd9c8fd95", "content-type": "application/x-amz-json-1.1", "content-length": "68", "date": "Thu, 31 Oct 2019 10:05:16 GMT"}, "RetryAttempts": 0}}
job_id: 34fa6597931e583f4d05bfee08bc0c8d
describe_topics_detection_job_result: {"TopicsDetectionJobProperties": {"JobId": "34fa6597931e583f4d05bfee08bc0c8d", "JobStatus": "SUBMITTED", "SubmitTime": {"$date": 1572516316641}, "InputDataConfig": {"S3Uri": "s3://yelpsuggest/las_vegas_reviews.csv", "InputFormat": "ONE_DOC_PER_LINE"}, "OutputDataConfig": {"S3Uri": "s3://yelpsuggest/las_vegas_topics.csv/844760149411-TOPICS-34fa6597931e583f4d05bfee08bc0c8d/output/output.tar.gz"}, "NumberOfTopics": 5, "DataAccessRoleArn": "arn:aws:iam::844760149411:role/yelp+suggest"}, "ResponseMetadata": 

### Code to download the file from amazon s3

In [0]:
s3r = boto3.resource('s3', aws_access_key_id='*******', aws_secret_access_key='******')
buck = s3r.Bucket('bucket name')
buck.download_file(remotefilename,localfilename)

### Here after downloading the files of Amazon comprehend output, we transformed the data using the easy MS excel "pivot tables" feature to get aggreted data.

##### Average ratings for each business under each topic.

In [0]:
finalres = pd.read_csv("drive/My Drive/Data Mining/finalres.csv",index_col="business_id")
finalres.head()

,varieties,ambience,management,service,value
business_id,,,,,
_0x7W6fizaPP76xNBxBLAQ,4.130178,4.187500,4.209302,3.948718,3.666667
_2fjiOHfu-S7fS2n4wfySw,4.225806,3.766667,4.125000,3.714286,2.000000
_3FxLxdg1unUQc3QqIgJuA,3.677291,3.704403,3.813559,3.800000,3.250000
_4tCeVrLRmI18BscMiHr7Q,4.275862,4.225806,4.400000,4.333333,4.333333
_6UaHEnDDbUla3Pa3VVu3Q,4.672727,4.680851,4.377778,4.846154,4.333333


##### Count of each business under each topic.

In [0]:
finalrescnt = pd.read_csv("drive/My Drive/Data Mining/finalrescnt.csv",index_col="business_id")
finalrescnt.head()

,varieties,ambience,management,service,value
business_id,,,,,
_0x7W6fizaPP76xNBxBLAQ,169,112,129,39,6
_2fjiOHfu-S7fS2n4wfySw,31,30,24,7,2
_3FxLxdg1unUQc3QqIgJuA,251,159,177,55,8
_4tCeVrLRmI18BscMiHr7Q,58,31,40,15,3
_6UaHEnDDbUla3Pa3VVu3Q,55,47,45,13,3


In [0]:
threshold = avg_topic_rating(finalres,finalrescnt)

In [0]:
threshold['varieties'][0]

3.825991741303466

### Example: for a given bussiness Id, we represent each topic as red card or green card in UI

In [0]:
id = "_3FxLxdg1unUQc3QqIgJuA"
if(finalres[finalres.index == id]['varieties'][0] < threshold['varieties'][0]):
  print("Red")
else:
  print("Green")  

Red


___________________________________________________________________________________

### Python trails for transformation instead of MS Excel "Pivot Table" feature

In [0]:
file=r'drive/My Drive/Dataset/Yelp_Reviews.parquet'
reviews=spark.read.parquet(file)
file1=r'drive/My Drive/Data Mining/restauranttokens2.csv/doc-topics.csv'
topics=spark.read.csv(file1,header=True)
# create a temporary view to be used in SQL queries
reviews.createOrReplaceTempView("reviews")
# create a temporary view to be used in SQL queries
topics.createOrReplaceTempView("topics")

In [0]:
def tokenize(c):
   return split(c,":")[1]

res = topics.withColumn('doc',tokenize(topics.docname)).drop("docname")

In [0]:
res = res.withColumn('doc',res.doc.cast(LongType()))

In [0]:
res.select(["doc","topic"]).orderBy("doc").show()

+---+-----+
|doc|topic|
+---+-----+
|  0|  002|
|  1|  001|
|  2|  000|
|  3|  000|
|  4|  002|
|  4|  000|
|  5|  000|
|  6|  002|
|  6|  004|
|  7|  000|
|  7|  001|
|  8|  000|
|  9|  002|
| 10|  002|
| 10|  004|
| 11|  000|
| 12|  001|
| 13|  002|
| 14|  003|
| 15|  000|
+---+-----+
only showing top 20 rows



In [0]:
from pyspark.sql.window import Window  
w = Window.orderBy("business_id") 
filtered_reviews = filtered_reviews.withColumn("id", row_number().over(w)-1)
filtered_reviews.filter(filtered_reviews['id']==0).show()

In [0]:
filtered_reviews.filter(filtered_reviews['id']==0).show()

+--------------------+--------------------+-------------------+------+---+
|         business_id|         review_text|              dates|rating| id|
+--------------------+--------------------+-------------------+------+---+
|--9e1ONYQuAa-CB_R...|Very busy and noi...|2015-10-23 13:23:34|   2.0|  0|
+--------------------+--------------------+-------------------+------+---+



In [0]:
filtered_reviews.count()

999453

In [0]:
res.printSchema()

root
 |-- topic: string (nullable = true)
 |-- proportion: string (nullable = true)
 |-- doc: long (nullable = true)



In [0]:
filtered_reviews.select("id").rdd.max()

Row(id=999452)

In [0]:
res.select("doc").distinct().count()

999453

In [0]:
# create a temporary view to be used in SQL queries
filtered_reviews.createOrReplaceTempView("filtered_reviews")

##### Note: Right outer join here (Perfect example!) 

In [0]:
query = """
SELECT filtered_reviews.id,filtered_reviews.business_id,res.topic,filtered_reviews.rating FROM res right outer join filtered_reviews on res.doc = filtered_reviews.id
"""
# convert the result to Pandas dataframe
merge = spark.sql(query)

In [0]:
merge.printSchema()

root
 |-- id: integer (nullable = true)
 |-- business_id: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- rating: double (nullable = true)



In [0]:
merge.select("business_id","topic").orderBy("business_id","topic").show()

+--------------------+-----+
|         business_id|topic|
+--------------------+-----+
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
|--9e1ONYQuAa-CB_R...|  000|
+--------------------+-----+
only showing top 20 rows



In [0]:
merge.count()

1471432

In [0]:
# create a temporary view to be used in SQL queries
merge.createOrReplaceTempView("merge")

query = """
SELECT business_id,topic,avg(rating) FROM merge group by business_id,topic order by business_id,topic
"""
# convert the result to Pandas dataframe
mergegrp = spark.sql(query)

In [0]:
mergegrp.count()

13331

In [0]:
# create a temporary view to be used in SQL queries
merge.createOrReplaceTempView("merge")

query = """
SELECT business_id,topic,count(rating) FROM merge group by business_id,topic order by business_id,topic
"""
# convert the result to Pandas dataframe
mergegrpcnt = spark.sql(query)

In [0]:
mergegrpcnt.count()

13331

In [0]:
mergegrpcnt.show()

+--------------------+-----+-------------+
|         business_id|topic|count(rating)|
+--------------------+-----+-------------+
|--9e1ONYQuAa-CB_R...|  000|          889|
|--9e1ONYQuAa-CB_R...|  001|          598|
|--9e1ONYQuAa-CB_R...|  002|          627|
|--9e1ONYQuAa-CB_R...|  003|          224|
|--9e1ONYQuAa-CB_R...|  004|           64|
|-1m9o3vGRA8IBPNvN...|  000|          147|
|-1m9o3vGRA8IBPNvN...|  001|           85|
|-1m9o3vGRA8IBPNvN...|  002|           94|
|-1m9o3vGRA8IBPNvN...|  003|           36|
|-1m9o3vGRA8IBPNvN...|  004|           10|
|-1vfRrlnNnNJ5boOV...|  000|           39|
|-1vfRrlnNnNJ5boOV...|  001|           31|
|-1vfRrlnNnNJ5boOV...|  002|           35|
|-1vfRrlnNnNJ5boOV...|  003|           12|
|-1vfRrlnNnNJ5boOV...|  004|            3|
|-3zffZUHoY8bQjGfP...|  000|          365|
|-3zffZUHoY8bQjGfP...|  001|          222|
|-3zffZUHoY8bQjGfP...|  002|          241|
|-3zffZUHoY8bQjGfP...|  003|           85|
|-3zffZUHoY8bQjGfP...|  004|           37|
+----------

#### After merging make sure this count is same as the business considered initially

In [0]:
mergegrp.select("business_id").distinct().count()

2680

In [0]:
mergegrpcnt.coalesce(1).write.option("header","true").csv("drive/My Drive/Data Mining/finalcnt.csv")

In [0]:
import pyspark.sql.functions as f 
merge.groupby("business_id").agg(f.collect_list("topic").alias("topics")).show()

+--------------------+--------------------+
|         business_id|              topics|
+--------------------+--------------------+
|--9e1ONYQuAa-CB_R...|[002, 000, 001, 0...|
|BMAflH82OUIMl2mHt...|[001, 000, 002, 0...|
|Cl-xl1vTUwHeaGgBx...|[001, 000, 000, 0...|
|Ipkl7KtlqxCe8bMMk...|[001, 000, 001, 0...|
|RSOinkOUpxm0mGw1I...|[002, 000, 003, 0...|
|_ixV2SWDy7w8jzEAH...|[004, 001, 001, 0...|
|dk1MV0MP32Xq-iBxz...|[000, 001, 001, 0...|
|nt2-Zk4FmGY2SYSDB...|[001, 002, 001, 0...|
|rbH0HuTcBvZBbiAnq...|[000, 003, 001, 0...|
|x1fx7C_tc064cFATj...|[000, 000, 001, 0...|
|MgSd4P3ATkiywbbra...|[000, 000, 001, 0...|
|G58YATMKnn-M-RUDW...|[001, 000, 002, 0...|
|5JNvfPfd6mcpwNoek...|[000, 001, 000, 0...|
|JJAwT9R6Fp2yyWHtW...|[002, 001, 000, 0...|
|L6qd3x0bHymyxub9d...|[001, 001, 000, 0...|
|PBZyumZ_0vSijc2zO...|[001, 002, 002, 0...|
|S6apFS5ghsQg69rcB...|[000, 001, 001, 0...|
|Sovgwq-E-n6wLqNh3...|[000, 001, 002, 0...|
|eJCDUkFa7wl-BHYKZ...|[001, 000, 002, 0...|
|mjhwLcuoenVW8oFhc...|[000, 001,

Still more transformations are required....... However since we have restricted to las vegas businesses / Restarants. Extracting output and working with pivot tables was easier.

### Trails : Below Functions Definitions -- for use in smaller datasets and for local LDA topic modelling.

In [0]:
# setup library imports
import io, time, json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import nltk
import string
from collections import Counter
from gensim import corpora, models
from nltk.tokenize import word_tokenize
import gensim
from textblob import TextBlob
from itertools import*
from collections import Counter

In [0]:
import logging
import boto3
from botocore.exceptions import ClientError


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
def categorical_reviews(category):
  filtered_business = business_df.filter(business_df['categories']==category)
  business_pandas = filtered_business.limit(100000).toPandas()
  business_list = list(business_pandas['business_id'])
  filtered_reviews = reviews_df.filter(reviews_df['business_id'].isin(business_list))
  reviews_pandas = filtered_reviews.limit(100000).toPandas()
  return reviews_pandas

#### Web scraping -- Takes a lot of time (If number of reviews are more for a particular restaurant)

In [0]:
def process(text, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ Normalizes case, handles punctuation and removes Stop words 
    Inputs:
        text: str: raw text
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs:
        list(str): tokenized text
    """
    # To remove any tags present in the review 
    soup = BeautifulSoup(text,'html.parser')
    text = soup.get_text()
    
    tspt = text.lower()
    tspt = tspt.replace("'s",'')
    tspt = tspt.replace("'",'')
    for ii in string.punctuation:
        tspt = tspt.replace(ii, ' ')
    tspt = nltk.word_tokenize(tspt) 
    
    # To remove the stop words 
    stopwords=nltk.corpus.stopwords.words('english')

    wrds = []
    for tt in tspt : 
        if tt not in stopwords:
            try :
                wrd = lemmatizer.lemmatize(tt)
            except :
                continue

            wrds.append( wrd)
    
    return wrds

In [0]:
def tokenize_review(df,field_name):
    lemm_text =  [process(row[field_name]) for ind, row in df.iterrows()]
    df = df.assign(reviews_token = lemm_text)
    return df 

In [0]:
def get_rare_words(processed_reviews,field_name):
    """ use the word count information across all reviews to come up with a feature list
    Inputs:
        processed_reviews: pd.DataFrame
    Outputs:
        list(str): list of rare words, sorted alphabetically.
    """
    
    out = []
    for ii in processed_reviews.iterrows() :
        out = out + ii[1][field_name]
    
    outfi = []
    of = Counter(out)
    for kk in of.keys() :
        if of[kk] == 1:
            outfi.append(kk)

    outfi = sorted(outfi)
    
    return outfi


In [0]:
def remove_rare(text, rare_wrds):
    """Removes rare words from the review tokens
    Inputs:
        text(list): tokenized review
        rare_wrds (list): rare words  
    Outputs:
        list of words, without the rare words.
    """
    output = []
    for wrd in text:
        if wrd not in rare_wrds:
            output.append(wrd)
    return output

In [0]:
def reviews_without_rare_words (df, token_field,  rare_words):
    """To get the tokenized review without the rare words
    Inputs: 
        df(dataframe): Reviews dataframe
        token_field (String) : field containing the tokenized reviews
        rare_words (list) : List of rarewords 
    Returns : 
        Dataframe with other column named 'reviews_without_rare_words' 
    """
    remove_rare_wrds = df[token_field].apply( lambda x: remove_rare(x, rare_wrds))
    df = df.assign(reviews_without_rare_words = remove_rare_wrds)
    return df 

In [0]:
def topic_model_prep(category):
  #for cat in toplist:
  df = categorical_reviews(category)
  df = tokenize_review(df,'review_text')
  rare_wrds = get_rare_words(df, 'reviews_token')
  df = reviews_without_rare_words(df,'reviews_token',rare_wrds)
  return df

In [0]:
df = categorical_reviews('Pizza')

In [0]:
df = tokenize_review(df,'review_text')

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [0]:
df.head()

,business_id,review_text,dates,rating,reviews_token
0,I4Nr-MVc26qWr08-S3Q1ow,The food is always good and the prices are rea...,2016-01-17 05:26:22,4.0,"[food, always, good, price, reasonable, althou..."
1,d_L-rfS1vT3JMzgCUGtiow,Pick any meat on the planet and the chef will ...,2016-07-25 03:57:19,5.0,"[pick, meat, planet, chef, make, mexican, styl..."
2,Sfc8Haz2Yri8Mo1L0E0TsA,"Great food, great service. Obviously fried chi...",2012-11-06 03:41:43,5.0,"[great, food, great, service, obviously, fried..."
3,Gyrez6K8f1AyR7dzW9fvAw,their pettuccine was fresh-made in the morning...,2013-12-28 22:28:08,5.0,"[pettuccine, fresh, made, morning, delicate, y..."
4,dm6sO_Y8JdKTE1ZM955yug,ended up here because Raku was closed and it r...,2014-12-17 19:04:33,5.0,"[ended, raku, closed, received, great, rating,..."


In [0]:
rare_wrds = get_rare_words(df, 'reviews_token')

In [0]:
filtered_business = business_df.filter(business_df['categories']=='Restaurants')
business_pandas = filtered_business.limit(100000).toPandas()
business_list = list(business_pandas['business_id'])
filtered_reviews = reviews_df.filter(reviews_df['business_id'].isin(business_list))
reviews_pandas = filtered_reviews.limit(100000).toPandas()

In [0]:
reviews_pandas.head()

,business_id,review_text,dates,rating
0,I4Nr-MVc26qWr08-S3Q1ow,The food is always good and the prices are rea...,2016-01-17 05:26:22,4.0
1,d_L-rfS1vT3JMzgCUGtiow,Pick any meat on the planet and the chef will ...,2016-07-25 03:57:19,5.0
2,Sfc8Haz2Yri8Mo1L0E0TsA,"Great food, great service. Obviously fried chi...",2012-11-06 03:41:43,5.0
3,Gyrez6K8f1AyR7dzW9fvAw,their pettuccine was fresh-made in the morning...,2013-12-28 22:28:08,5.0
4,dm6sO_Y8JdKTE1ZM955yug,ended up here because Raku was closed and it r...,2014-12-17 19:04:33,5.0


In [0]:
def create_LDA_model(df, review_field, no_topics, no_passes):
    """creates LDA model 
    Inputs:
        df (dataframe) : Reviews dataframe
        review_field (String) : tokenized reviews column
        no_topics (integer) : number of topics for LDA model
        no_passes(integer) : number of passes for LDA model
    Returns :
            tuple (corpusm, gensim.models.ldamodel.LdaModel, Dictionary)  
    """
    text = list(df[review_field])
    dictionary = corpora.Dictionary(text)
    corpus = [dictionary.doc2bow(txt) for txt in text]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=no_topics, id2word = dictionary, passes=no_passes)
    return (corpus, ldamodel, dictionary)

In [0]:
np.random.seed(1)
def train_test_split(df, train_size):
    """Splits the dataframe to train and test according to the given size 
    Inputs: 
        df(dataframe) : Reviews dataframe
        train_size(float) : ratio of train size
    Returns: 
        tuple(train,test) dataframes
    """
    random_perm = np.random.permutation(len(df))
    train = df.iloc[random_perm[:int(train_size*len(df))]]
    test = df.iloc[random_perm[int(train_size*len(df)):]]
    return (train,test)

In [0]:
pizza_train,pizza_test = train_test_split(df,0.9)

In [0]:
pizza_corpus, pizza_train_model, pizza_dictionary = create_LDA_model(pizza_train, 'reviews_token', 6, 10)

In [0]:
pizza_train_model.print_topics(num_topics = 6 ,num_words = 10)

[(0,
  '0.010*"dish" + 0.010*"sushi" + 0.009*"good" + 0.008*"salad" + 0.008*"dessert" + 0.008*"shrimp" + 0.007*"roll" + 0.007*"steak" + 0.006*"dinner" + 0.006*"ordered"'),
 (1,
  '0.017*"chicken" + 0.016*"cheese" + 0.016*"burger" + 0.014*"good" + 0.014*"pizza" + 0.012*"sandwich" + 0.012*"fry" + 0.011*"egg" + 0.008*"ordered" + 0.008*"like"'),
 (2,
  '0.036*"great" + 0.035*"food" + 0.025*"place" + 0.024*"service" + 0.015*"good" + 0.013*"time" + 0.013*"amazing" + 0.013*"vega" + 0.011*"best" + 0.011*"back"'),
 (3,
  '0.016*"good" + 0.014*"place" + 0.014*"rice" + 0.013*"chicken" + 0.013*"food" + 0.011*"soup" + 0.011*"noodle" + 0.010*"dish" + 0.010*"like" + 0.008*"beef"'),
 (4,
  '0.015*"food" + 0.014*"u" + 0.012*"time" + 0.012*"order" + 0.010*"service" + 0.008*"table" + 0.008*"back" + 0.008*"one" + 0.008*"came" + 0.008*"get"'),
 (5,
  '0.019*"food" + 0.015*"good" + 0.014*"buffet" + 0.013*"place" + 0.011*"vega" + 0.010*"like" + 0.010*"get" + 0.009*"price" + 0.008*"go" + 0.007*"one"')]

In [0]:
def get_topic_distribution(ldamodel, dictionary, tokenized_text):
    """
    Returns topic distribution for the given tokenized review
    """
    bag_of_words = dictionary.doc2bow(tokenized_text)
    topic_dist = ldamodel[bag_of_words]
    return topic_dist

In [0]:
def topic_ratings_all(dictionary,ldamodel, df, review_field, no_topics ):

    df = df.copy()
    text = list(df[review_field])
    corpus = [dictionary.doc2bow(txt) for txt in text]

    topic_ratings = [ldamodel[i] for i in corpus]
    names = ['Topic '+ str(i) for i in range(no_topics)]
    for i in range(len(names)):
        df[names[i]] =  [np.nan]*len(df)
    for i , j in enumerate(df.index):
        rating = df.ix[j]['rating']
        for k in topic_ratings[i]:
            df.set_value( j, 'Topic ' + str(k[0]), rating )
    df_topic_ratings = df.groupby('business_id').mean()[names]
    df_topic_counts = df.groupby('business_id').count()[names]
    return df_topic_ratings , df_topic_counts

In [0]:
pizza_ratings, pizza_counts = topic_ratings_all(pizza_dictionary, pizza_train_model, df, 'reviews_token', 6)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [0]:
pizza_ratings.head()

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
business_id,,,,,,,,,,
-9YyInW1wapzdNZrhQJ9dg,2.696078,2.529412,3.000000,2.093750,2.345794,2.494382,NaN,NaN,NaN,NaN
-AGdGGCeTS-njB_8GkUmjQ,4.166667,4.166667,4.250000,4.250000,4.000000,4.000000,NaN,NaN,NaN,NaN
-BmqghX1sv7sgsxOIS2yAg,4.020979,4.076923,4.160839,4.033898,3.901408,4.033784,NaN,NaN,NaN,NaN
-U7tvCtaraTQ9b0zBhpBMA,2.744914,2.728938,3.055446,2.566298,2.495007,2.624438,NaN,NaN,NaN,NaN
-xX-Vjol_fwWaN2mn1F1wg,4.285714,3.444444,3.857143,3.714286,3.071429,3.352941,NaN,NaN,NaN,NaN


In [0]:
pizza_counts.head()

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
business_id,,,,,,,,,,
-9YyInW1wapzdNZrhQJ9dg,102,102,75,64,107,178,0,0,0,0
-AGdGGCeTS-njB_8GkUmjQ,6,6,8,8,5,5,0,0,0,0
-BmqghX1sv7sgsxOIS2yAg,143,78,143,118,142,148,0,0,0,0
-U7tvCtaraTQ9b0zBhpBMA,639,546,505,362,701,1113,0,0,0,0
-xX-Vjol_fwWaN2mn1F1wg,7,18,14,7,14,17,0,0,0,0


In [0]:
def avg_topic_rating (ratings_df , counts_df):
    """
    Gets the average rating per topic 
    """
    ratings_array = np.array(ratings_df)
    counts_array = np.array(counts_df)
    mult = np.multiply(ratings_array,counts_array)
    sum_ratings = np.sum(np.nan_to_num(mult),axis=0)
    avg_ratings = sum_ratings / np.sum(counts_array,axis = 0)
    avg_ratings_df = pd.DataFrame(avg_ratings.reshape(1, len(avg_ratings)) ,
                                  columns= ratings_df.columns, index=['Avg Topic Rating'])
    return avg_ratings_df

In [0]:
avg_topic_rating(pizza_ratings,pizza_counts)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
Avg Topic Rating,3.845176,3.778254,4.096603,3.79759,3.504256,3.628002,NaN,NaN,NaN,NaN
